In [13]:
#load "load.fsx"

open Newtonsoft.Json

open FSharpPlus
open FSharpPlus.Data
open Informedica.Utils.Lib.BCL

open Informedica.OpenAI.Lib


In [14]:
let test model =
    [
        for (text, exp) in Texts.testUnitTexts do
            let act, msgs =
                [
                    Prompts.System.systemDoseQuantityExpert text
                    |> Message.system
                ]
                |> State.run
                    (Ollama.Extract.frequencies model text)

            msgs |> List.iter Message.print

            1 // need to implement: if act = exp then 1 else 0
            , (act, exp)
            , text
    ]

In [15]:
let calcScore = List.sumBy (fun (s, _, _) -> s)


let testResults =
    [
        Ollama.Models.llama2
//        Ollama.Models.gemma
        Ollama.Models.openhermes
        Ollama.Models.mistral
//        Ollama.Models.``llama-pro``
//        Ollama.Models.``openchat:7b``
//        Ollama.Models.``llama2:13b-chat``
    ]
    |> List.map (fun model -> 
        printf $"- Testing: {model}: "
        let s = model |> test
        printfn $"score: {s |> calcScore}"
        model, s
    )    

- Testing: llama2: 
## System:
You are an expert on medication prescribing, preparation and administration. You will give
exact answers. If there is no possible answer return an empty string.
You will be asked to extract structured information from the following text between ''':

'''
alprazolam
6 jaar tot 18 jaar Startdosering: 0,125 mg/dag, éénmalig. Onderhoudsdosering: Op geleide van klinisch beeld verhogen met stappen van 0,125-0,25 mg/dosis tot max 0,05 mg/kg/dag in 3 doses. Max: 3 mg/dag. Advies inname/toediening: De dagdosis indien mogelijk verdelen over 3 doses.Bij plotselinge extreme slapeloosheid: alleen voor de nacht innemen; dosering op geleide van effect ophogen tot max 0,05 mg/kg, maar niet hoger dan 3 mg/dag.De effectiviteit bij de behandeling van acute angst is discutabel.
'''

ONLY respond if the response is actually present in the text. If the response cannot be extracted
respond with an empty string for a textfield or zero for a number field.

Respond in JSON


## Qu

In [5]:
testResults    
|> List.maxBy (fun (model, xs) -> xs |> calcScore)
|> fun (m, s) -> printfn $"\n\n## And the winner is: {m} with a high score: {s |> calcScore} from {Texts.testUnitTexts |> List.length}"

testResults
|> List.collect (fun (m, xs) -> xs)
|> List.groupBy (fun (_, _, t) -> t)
|> List.map (fun (txt, s) -> s |> calcScore, txt)
|> List.sortBy fst
|> List.iter (fun (s, t) ->
    printfn $"## Score: {s}"
    printfn $"## Text:\n{t}"
)



## And the winner is: llama2 with a high score: 6 from 6
## Score: 4
## Text:

alprazolam
6 jaar tot 18 jaar Startdosering: 0,125 mg/dag, éénmalig. Onderhoudsdosering: Op geleide van klinisch beeld verhogen met stappen van 0,125-0,25 mg/dosis tot max 0,05 mg/kg/dag in 3 doses. Max: 3 mg/dag. Advies inname/toediening: De dagdosis indien mogelijk verdelen over 3 doses.Bij plotselinge extreme slapeloosheid: alleen voor de nacht innemen; dosering op geleide van effect ophogen tot max 0,05 mg/kg, maar niet hoger dan 3 mg/dag.De effectiviteit bij de behandeling van acute angst is discutabel.

## Score: 4
## Text:

acetylsalicylzuur
1 maand tot 18 jaar Startdosering:Acetylsalicylzuur: 30 - 50 mg/kg/dag in 3 - 4 doses. Max: 3.000 mg/dag.

## Score: 4
## Text:

paracetamol
Oraal: Bij milde tot matige pijn en/of koorts: volgens het Kinderformularium van het NKFK bij een leeftijd van 1 maand–18 jaar: 10–15 mg/kg lichaamsgewicht per keer, zo nodig 4×/dag, max. 60 mg/kg/dag en max. 4 g/dag.

## S